In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
%autoreload
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.isp import fourdirection2xy
import torch
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []

print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
        
all_data = valdata + traindata


In [ ]:
%autoreload
from tianmoucv.proc.reconstruct.tiny_unet import Reconstrutor_NN

device = torch.device('cuda:3')
reconstructor = Reconstrutor_NN(ckpt_path=None,_optim=False).to(device)

# 融合图像

In [ ]:
%autoreload
import torch.nn as nn
import math,time

def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 360        
    ftmax = 1
    ftmin = 0
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft[ft>1]=1
        ft[ft<0]=0
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()


key_list = ['train_exam_fan5']
for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500,matchkey=key,speedUpRate=1)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    img_list = []
    count = 0
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 3:
            F0 = sample['F0'][0,...]
            F1 = sample['F1'][0,...]
            tsdiff = sample['tsdiff'][0,...]
            #正向重建
            reconstructed_b1 = reconstructor(F0.to(device),tsdiff.to(device), t=-1).float()
            inverse_tsdiff = torch.zeros_like(tsdiff)
            timelen = tsdiff.shape[1]
            for t in range(timelen):
                if t < timelen-1:
                    inverse_tsdiff[0,t,...] = tsdiff[0,timelen-t-1,...] * -1
                inverse_tsdiff[1,t,...] = tsdiff[1,timelen-t-1,...]
                inverse_tsdiff[2,t,...] = tsdiff[2,timelen-t-1,...]
            
            #逆向重建
            reconstructed_b2 = reconstructor(F1.to(device),inverse_tsdiff.to(device), t=-1).float()
            #求平均压制TD噪声
            reconstructed_b = torch.zeros_like(reconstructed_b1)
            
            for t in range(timelen):
                reconstructed_b[t,...] = (reconstructed_b1[t,...]+reconstructed_b2[timelen-t-1])/2

            #最后一帧可以扔掉，或者跟下一次的重建的第0帧做个平均，降低一些闪烁感
            for t in range(timelen-1):
                reconstructed = reconstructed_b[t,...].cpu()
                showim = torch.cat([F0,reconstructed.permute(1,2,0)],dim=0).numpy()
                w = 640
                h = 320
                count += 1
                cv2.putText(showim,"COP+"+str(t)+"*AOP,IDX:"+str(count),(12,36+1*h),cv2.FONT_HERSHEY_SIMPLEX,0.5,(1,1,1),4)
                if t==12:
                    plt.figure(figsize=(4,8))
                    plt.subplot(1,1,1)  
                    plt.imshow(showim)
                    plt.show()
                img_list.append(showim[...,[2,1,0]])
        else:
            break
    images_to_video(img_list,'./viz_'+key+'.mp4',size=(640,640),Flip=True)

In [ ]:
def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 50        
    ftmax = 1
    ftmin = 0
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft[ft>1]=1
        ft[ft<0]=0
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()
images_to_video(img_list,'./viz_'+key+'.mp4',size=(640,640),Flip=True)